In [14]:
import os
import json
import mediapipe as mp

In [15]:
videos = os.listdir("dataset/videos")

In [11]:
with open("dataset/WLASL_v0.3.json", 'r') as file:
    data_map = json.load(file)

In [8]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [16]:
VIDEOS_DIR = "dataset/videos"

In [17]:
import cv2

In [26]:
word_coordinates = {}

In [ ]:
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    for word in data_map:
        word_coordinates[word["gloss"]] = []
        video_id = word["instances"][0]["video_id"]
        video_path = os.path.join(VIDEOS_DIR, video_id + ".mp4")
        video = cv2.VideoCapture(video_path)
        while video.isOpened():
                ret, frame = video.read()

                if not ret:
                    break

                # Resize the frame to 800x750
                frame = cv2.resize(frame, (800, 750))

                # Convert the frame to RGB
                image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # Detections
                results = hands.process(image_rgb)

                if results.multi_hand_landmarks:
                    # Initialize hand coordinates
                    hand_coordinates = []

                    frame_hands = {
                        "Left Hand Coordinates": [],
                        "Right Hand Coordinates": []
                    }

                    # Process each detected hand
                    for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                        # Get the handedness classification from MediaPipe
                        handedness = results.multi_handedness[idx].classification[0].label
                        hand = hand_landmarks.landmark

                        coordinates = []
                        # Store coordinates and joint index in the hand coordinates list
                        for joint_id, landmark in enumerate(hand):
                            x, y, z = landmark.x, landmark.y, landmark.z
                            joint_data = {
                                "Joint Index": joint_id,
                                "Coordinates": [x, y, z]
                            }
                            coordinates.append(joint_data)
                        frame_hands[f"{handedness} Hand Coordinates"] = coordinates
                    word_coordinates[word["gloss"]].append(frame_hands)


In [28]:
with open('coordinates.json', 'w') as fp:
    json.dump(word_coordinates, fp)